# Initial Setup

In [1]:
import os
import json
import logging
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

logger = logging.getLogger()
fhandler = logging.FileHandler(filename="chatbot_tools.log", mode="a")
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [2]:
# Local bot
OLLAMA_API_KEY = "ollama"
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama_client = OpenAI(api_key=OLLAMA_API_KEY, base_url=OLLAMA_BASE_URL)
ollama_model = "llama3.2"

# Online bot
load_dotenv(override=True)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "type-your-api-here")
GOOGLE_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_client = OpenAI(api_key=GOOGLE_API_KEY, base_url=GOOGLE_BASE_URL)
gemini_model = "gemini-2.0-flash"

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "type-your-api-here")
gpt_client = OpenAI()
gpt_model = "gpt-4o-mini"

# Chatbot Tool(s)

In [3]:
# Toy dataset
ticket_prices = {
    "batam": "Rp1.932.020",
    "bali": "Rp1.114.920",
    "gorontalo": "2.065.320",
    "lampung": "Rp595.700"
}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    logging.debug(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

## Putting Tool(s) into UI

In [4]:
# 1. Create required dict structure, description should be in plain English
get_ticket_price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to"
            }
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [5]:
# 2. Create a list of Tools for OpenAI API
tools = [{"type": "function", "function": get_ticket_price_function}]

# Prompting

In [6]:
# Unpack tools request and decide which tools to use
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    logging.debug(f"Tool call: {tool_call}")
    
    arguments = json.loads(tool_call.function.arguments)
    destination_city = arguments.get('destination_city')
    price = get_ticket_price(destination_city)
    
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": destination_city, "price": price}),
        "tool_call_id": tool_call.id
    }
    return response, destination_city

In [7]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

def chat(message, history, client=gpt_client, model=gpt_model):
    """Gradio dedicated function for chatbot"""
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        temperature=0.3
    )

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response, destination_city = handle_tool_call(message) # unpack tools request messages
        messages.append(message)
        messages.append(response)
        response = client.chat.completions.create(model=model, messages=messages, temperature=0.3) # sending back messages to gpt
    
    return response.choices[0].message.content

# UI

In [8]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
